In [1]:
import pickle
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy
import cartopy.feature as cfeature
from datetime import datetime
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 


In [2]:
with open('/Users/acheung/data/intermediates/mrms_list_indiv.pkl', 'rb') as file:
    # Load the data from the file
    interpolated_mrms_list = pickle.load(file)
    
with open('/Users/acheung/data/intermediates/lat_lon_list_indiv.pkl', 'rb') as file:
    # Load the data from the file
    lat_lon_arr_list = pickle.load(file)
    
with open('/Users/acheung/data/intermediates/imerg_list_indiv.pkl', 'rb') as file:
    # Load the data from the file
    imerg_precip_list = pickle.load(file)
    
with open('/Users/acheung/data/intermediates/time_strings_list_indiv.pkl', 'rb') as file:
    # Load the data from the file
    time_strings_list = pickle.load(file)
    
with open('/Users/acheung/data/intermediates/datetimes_indiv.pkl', 'rb') as file:
    # Load the data from the file
    individual_times_list = pickle.load(file)

In [10]:
len(imerg_precip_list[23])

24

In [15]:
from multiprocessing import Pool

# Stack overflow: https://stackoverflow.com/questions/29857498/how-to-apply-a-function-to-a-2d-numpy-array-with-multiprocessing

def percentile_calc(case_now,fraction_ind_0,fraction_ind_1):
    if case_now[fraction_ind_0][fraction_ind_1].isnull() == True:
        current_point_percentile = np.nan
    if case_now[fraction_ind_0][fraction_ind_1].isnull() == False:
        current_point_percentile = (case_now < case_now[fraction_ind_0][fraction_ind_1]).sum()/(~case_now.isnull()).sum()
    return current_point_percentile
    
def splat_f(args):
    return percentile_calc(*args)

def parallel(M, N,case_now):
    result = pool.map(splat_f, ((case_now,fraction_ind_0,fraction_ind_1) for fraction_ind_0 in range(M) for fraction_ind_1 in range(N)))

    return np.array(result).reshape(M, N)

pool = Pool(8)


In [16]:
colors_list = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

In [17]:
for storm_plotter in range(len(imerg_precip_list)):
    for indiv_images in range(len(imerg_precip_list[storm_plotter])):
        case_now_pre  = imerg_precip_list[storm_plotter][indiv_images]
        case_now_xr = xr.DataArray(case_now_pre)/25.4
        M = case_now_xr.shape[0]
        N = case_now_xr.shape[1]
        fractions_matrix = parallel(M,N,case_now_xr)
        
        fig = plt.figure()
        ax=plt.axes(projection=ccrs.PlateCarree())
        ax.add_feature(cfeature.STATES)
        ax.add_feature(cfeature.COASTLINE)
        f_plot = ax.contourf(lat_lon_arr_list[storm_plotter][1],lat_lon_arr_list[storm_plotter][0],
                             case_now_xr,levels=np.arange(0,np.nanpercentile(case_now_xr,100) +
                                                          float('%.2g' % (np.nanpercentile(case_now_xr,100)/10)),
                                                          float('%.2g' % (np.nanpercentile(case_now_xr,100)/10))))
        plt.colorbar(f_plot,label='1 h Precipitation (in)')
        c_plot = ax.contour(lat_lon_arr_list[storm_plotter][1],lat_lon_arr_list[storm_plotter][0],
                            fractions_matrix,colors=colors_list,levels=[0.75,0.9,0.95,0.99])
        ax.clabel(c_plot,colors=colors_list)
        ts = (individual_times_list[storm_plotter][indiv_images]  - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
        ax.set_title('IMERG Precipitation\n' + str(datetime.utcfromtimestamp(ts)))
        plt.savefig('/Users/acheung/data/Figures/Individual_Percentile_Plots/IMERG_Plot_'+
                    str(datetime.utcfromtimestamp(ts)) + '_Storm_' + str(storm_plotter) + '.png')
        plt.close()
        print((storm_plotter,len(imerg_precip_list)),(indiv_images,len(imerg_precip_list[storm_plotter])))


(0, 42) (0, 42)
(0, 42) (1, 42)
(0, 42) (2, 42)
(0, 42) (3, 42)
(0, 42) (4, 42)
(0, 42) (5, 42)
(0, 42) (6, 42)
(0, 42) (7, 42)
(0, 42) (8, 42)
(0, 42) (9, 42)
(0, 42) (10, 42)
(0, 42) (11, 42)
(0, 42) (12, 42)
(0, 42) (13, 42)
(0, 42) (14, 42)
(0, 42) (15, 42)
(0, 42) (16, 42)
(0, 42) (17, 42)
(0, 42) (18, 42)
(0, 42) (19, 42)
(0, 42) (20, 42)
(0, 42) (21, 42)
(0, 42) (22, 42)
(0, 42) (23, 42)
(0, 42) (24, 42)
(0, 42) (25, 42)
(0, 42) (26, 42)
(0, 42) (27, 42)
(0, 42) (28, 42)
(0, 42) (29, 42)
(0, 42) (30, 42)
(0, 42) (31, 42)
(0, 42) (32, 42)
(0, 42) (33, 42)
(0, 42) (34, 42)
(0, 42) (35, 42)
(0, 42) (36, 42)
(0, 42) (37, 42)
(0, 42) (38, 42)
(0, 42) (39, 42)
(0, 42) (40, 42)
(0, 42) (41, 42)
(1, 42) (0, 12)
(1, 42) (1, 12)
(1, 42) (2, 12)
(1, 42) (3, 12)
(1, 42) (4, 12)
(1, 42) (5, 12)
(1, 42) (6, 12)
(1, 42) (7, 12)
(1, 42) (8, 12)
(1, 42) (9, 12)
(1, 42) (10, 12)
(1, 42) (11, 12)
(2, 42) (0, 24)
(2, 42) (1, 24)
(2, 42) (2, 24)
(2, 42) (3, 24)
(2, 42) (4, 24)
(2, 42) (5, 24)
(2, 42

In [18]:
ts

1668142800.0

In [ ]:
# fig = plt.figure()
# ax=plt.axes(projection=ccrs.PlateCarree())
# ax.add_feature(cfeature.STATES)
# ax.add_feature(cfeature.COASTLINE)
# f_plot = ax.contourf(lat_lon_arr_list[storm_plotter][1],lat_lon_arr_list[storm_plotter][0],
#                      case_now_xr,levels=np.arange(0,np.nanpercentile(case_now_xr,100) +
#                                                   np.round((np.nanpercentile(case_now_xr,100) + 0.01)/10,2),
#                                                   np.round((np.nanpercentile(case_now_xr,100) + 0.01)/10,2)))
# plt.colorbar(f_plot)
# c_plot = ax.contour(lat_lon_arr_list[storm_plotter][1],lat_lon_arr_list[storm_plotter][0],
#                     fractions_matrix,colors='white',levels=[0.75,0.9,0.95,0.99])
# ax.clabel(c_plot,colors='white')
# ax.set_title('IMERG Precipitation')

